<a href="https://colab.research.google.com/github/selcuk-yalcin/Convolutional-Neural-Network/blob/main/Densenet_data_Cifar_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
# ============================================================
# DenseNet Mini Example with CIFAR-10
# ============================================================
# Covers: Bottleneck Layer → Dense Block → Transition Layer
# Uses: real CIFAR-10 images
# ============================================================

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

In [23]:
# ============================================================
# Step 1: Define the Bottleneck Layer
# ============================================================

class BottleneckLayer(nn.Module):
    def __init__(self, in_channels, growth_rate):
        super(BottleneckLayer, self).__init__()
        inner_channels = 4 * growth_rate

        self.bn1 = nn.BatchNorm2d(in_channels)
        self.relu1 = nn.ReLU(inplace=True)
        self.conv1 = nn.Conv2d(in_channels, inner_channels, kernel_size=1, bias=False)

        self.bn2 = nn.BatchNorm2d(inner_channels)
        self.relu2 = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(inner_channels, growth_rate, kernel_size=3, padding=1, bias=False)

    def forward(self, x):
        out = self.conv1(self.relu1(self.bn1(x)))
        out = self.conv2(self.relu2(self.bn2(out)))
        out = torch.cat([x, out], 1)  # Dense connection
        return out

In [22]:
# ============================================================
# Step 2: Define a Dense Block
# ============================================================

class DenseBlock(nn.Module):
    def __init__(self, num_layers, in_channels, growth_rate):
        super(DenseBlock, self).__init__()
        self.layers = nn.ModuleList()
        for i in range(num_layers):
            layer = BottleneckLayer(in_channels + i * growth_rate, growth_rate)
            self.layers.append(layer)

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

In [21]:
# ============================================================
# Step 3: Define the Transition Layer
# ============================================================

class TransitionLayer(nn.Module):
    def __init__(self, in_channels, compression=0.5):
        super(TransitionLayer, self).__init__()
        out_channels = int(in_channels * compression)
        self.bn = nn.BatchNorm2d(in_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1, bias=False)
        self.pool = nn.AvgPool2d(2, stride=2)

    def forward(self, x):
        x = self.conv(self.relu(self.bn(x)))
        x = self.pool(x)
        return x

In [20]:
# ============================================================
# Step 4: Define a Mini DenseNet for CIFAR-10
# ============================================================

class MiniDenseNet(nn.Module):
    def __init__(self, in_channels=3, growth_rate=20, num_classes=10):
        super(MiniDenseNet, self).__init__()
        self.initial_conv = nn.Conv2d(in_channels, 24, kernel_size=3, padding=1, bias=False)
        self.block1 = DenseBlock(4, 24, growth_rate)
        self.trans1 = TransitionLayer(24 + 4 * growth_rate)
        self.block2 = DenseBlock(4, int((24 + 4 * growth_rate) * 0.5), growth_rate)
        self.trans2 = TransitionLayer(int((24 + 4 * growth_rate) * 0.5) + 4 * growth_rate)
        self.classifier = nn.Linear(int(((24 + 4 * growth_rate) * 0.5 + 4 * growth_rate) * 0.5), num_classes)

    def forward(self, x):
        x = self.initial_conv(x)
        x = self.block1(x)
        x = self.trans1(x)
        x = self.block2(x)
        x = self.trans2(x)

        x = F.adaptive_avg_pool2d(x, (1, 1))
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

In [24]:
# ============================================================
# Step 5: Load CIFAR-10 Dataset
# ============================================================

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

In [ ]:
# ============================================================
# Step 6: Train on CIFAR-10
# ============================================================

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MiniDenseNet().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0005)

print("\n Starting training...\n")

for epoch in range(10):  # small demo: 20 epochs
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(trainloader):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:
            print(f"[Epoch {epoch+1}, Batch {i+1}] Loss: {running_loss/100:.3f}")
            running_loss = 0.0

print("\n Training complete!")


 Starting training...

[Epoch 1, Batch 100] Loss: 1.937
[Epoch 1, Batch 200] Loss: 1.638
[Epoch 1, Batch 300] Loss: 1.513
[Epoch 1, Batch 400] Loss: 1.423
[Epoch 1, Batch 500] Loss: 1.351
[Epoch 1, Batch 600] Loss: 1.323
[Epoch 1, Batch 700] Loss: 1.273
[Epoch 2, Batch 100] Loss: 1.178
[Epoch 2, Batch 200] Loss: 1.116
[Epoch 2, Batch 300] Loss: 1.099
[Epoch 2, Batch 400] Loss: 1.073
[Epoch 2, Batch 500] Loss: 1.051
[Epoch 2, Batch 600] Loss: 1.026
[Epoch 2, Batch 700] Loss: 1.006
[Epoch 3, Batch 100] Loss: 0.941
[Epoch 3, Batch 200] Loss: 0.946
[Epoch 3, Batch 300] Loss: 0.934
[Epoch 3, Batch 400] Loss: 0.908
[Epoch 3, Batch 500] Loss: 0.892
[Epoch 3, Batch 600] Loss: 0.883
[Epoch 3, Batch 700] Loss: 0.869
[Epoch 4, Batch 100] Loss: 0.849
[Epoch 4, Batch 200] Loss: 0.842
[Epoch 4, Batch 300] Loss: 0.798
[Epoch 4, Batch 400] Loss: 0.778


In [19]:
# ============================================================
# Step 7: Evaluate on Test Data
# ============================================================

correct, total = 0, 0
model.eval()
with torch.no_grad():
    for inputs, labels in testloader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"\n Test Accuracy: {100 * correct / total:.2f}%")


 Test Accuracy: 73.96%
